In [ ]:
import os
os.chdir("..")

In [ ]:
from src.data.utils import DiffReg
import geopandas as gpd
from shapely import wkt
import polars as pl

dr = DiffReg()

In [ ]:
dr.usqcew_data()

In [ ]:
dr.regular_data(naics="72-food", foreign=True)